In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
import pandas as pd

In [2]:
#Conexión a la base de datos local
def crear_conexion():
    return mysql.connector.connect(
    host="localhost",
    user="root",
    password="Maxi1",
    database="Bienes_raices"
)

**CRUD**

In [3]:
#Crear propietario para luego crear una propiedad 
def crear_propietario(nombre, direccion, contacto):
    conexion = crear_conexion()
    cursor = conexion.cursor()
    sql = "INSERT INTO Propietario (Nombre, Direccion, Contacto) VALUES (%s, %s, %s)"
    cursor.execute(sql, (nombre, direccion, contacto))
    conexion.commit()
    
    # Se obtiene id del propietario
    id_propietario = cursor.lastrowid
    
    cursor.close()
    conexion.close()
    return id_propietario

In [4]:
def ingresar_propiedad(tipo, estado, operativa, nombre_propietario, direccion_propietario, contacto_propietario, nombre_propiedad, direccion_propiedad, contacto_propiedad):

    id_propietario = crear_propietario(nombre_propietario, direccion_propietario, contacto_propietario)
    
    #Conexion a la base de datos
    conexion = crear_conexion()
    cursor = conexion.cursor()
    
    #consulta para ingresar la propiedad
    sql = """
    INSERT INTO Propiedad (Id_Tipo, Id_Estado, Id_Operatoria_Comercial, Id_Propietario, Nombre, Direccion, Contacto) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    valores = (tipo, estado, operativa, id_propietario, nombre_propiedad, direccion_propiedad, contacto_propiedad)
    
    cursor.execute(sql, valores)
    conexion.commit()
    
    cursor.close()
    conexion.close()
    print("Propiedad ingresada exitosamente con nuevo propietario.")

In [5]:
def modificar_propiedad(id_propiedad, id_tipo=None, id_estado=None, id_operatoria_comercial=None, id_propietario=None, nombre=None, direccion=None, contacto=None):
    conexion = crear_conexion()
    cursor = conexion.cursor()
    sql = "UPDATE Propiedad SET"
    valores = []

    
    if id_tipo is not None:
        sql += " Id_Tipo = %s,"
        valores.append(id_tipo)
    if id_estado is not None:
        sql += " Id_Estado = %s,"
        valores.append(id_estado)
    if id_operatoria_comercial is not None:
        sql += " Id_Operatoria_Comercial = %s,"
        valores.append(id_operatoria_comercial)
    if id_propietario is not None:
        sql += " Id_Propietario = %s,"
        valores.append(id_propietario)
    if nombre is not None:
        sql += " Nombre = %s,"
        valores.append(nombre)
    if direccion is not None:
        sql += " Direccion = %s,"
        valores.append(direccion)
    if contacto is not None:
        sql += " Contacto = %s,"
        valores.append(contacto)

    #Se elimina la última coma y se agrega la condición WHERE.
    sql = sql.rstrip(',') + " WHERE Id_Propiedad = %s"
    valores.append(id_propiedad)

    cursor.execute(sql, valores)
    conexion.commit()
    cursor.close()
    conexion.close()
    print(f"Propiedad {id_propiedad} modificada exitosamente.")

In [6]:
#Eliminar una propiedad
def eliminar_propiedad(id_propiedad):
    conexion = crear_conexion()
    cursor = conexion.cursor()
    sql = "DELETE FROM Propiedad WHERE Id_Propiedad = %s"
    cursor.execute(sql, (id_propiedad,))
    conexion.commit()
    cursor.close()
    conexion.close()
    print(f"Propiedad {id_propiedad} eliminada exitosamente.")

**Reportes**

In [7]:
#Propiedades Totales
def listar_propiedades_totales():
    conexion = crear_conexion()
    sql = """
    SELECT p.*, t.Nombre_Tipo, e.Nombre_Estado, o.Nombre_Operatoria_Comercial 
    FROM Propiedad p
    JOIN Tipo t ON p.Id_Tipo = t.Id_Tipo
    JOIN Estado e ON p.Id_Estado = e.Id_Estado
    JOIN OperatoriaComercial o ON p.Id_Operatoria_Comercial = o.Id_Operatoria_Comercial
    """
    df = pd.read_sql(sql, conexion)
    conexion.close()
    return df

In [8]:
#Propiedades Disponibles para la Venta
def listar_propiedades_disponibles_venta():
    conexion = crear_conexion()
    sql = """
    SELECT p.*, t.Nombre_Tipo, e.Nombre_Estado, o.Nombre_Operatoria_Comercial 
    FROM Propiedad p
    JOIN Tipo t ON p.Id_Tipo = t.Id_Tipo
    JOIN Estado e ON p.Id_Estado = e.Id_Estado
    JOIN OperatoriaComercial o ON p.Id_Operatoria_Comercial = o.Id_Operatoria_Comercial
    WHERE e.Nombre_Estado = 'Disponible' AND o.Nombre_Operatoria_Comercial = 'Venta'
    """
    df = pd.read_sql(sql, conexion)
    conexion.close()
    return df

In [9]:
#Propiedades Disponibles para Alquiler
def listar_propiedades_disponibles_alquiler():
    conexion = crear_conexion()
    sql = """
    SELECT p.*, t.Nombre_Tipo, e.Nombre_Estado, o.Nombre_Operatoria_Comercial 
    FROM Propiedad p
    JOIN Tipo t ON p.Id_Tipo = t.Id_Tipo
    JOIN Estado e ON p.Id_Estado = e.Id_Estado
    JOIN OperatoriaComercial o ON p.Id_Operatoria_Comercial = o.Id_Operatoria_Comercial
    WHERE e.Nombre_Estado = 'Disponible' AND o.Nombre_Operatoria_Comercial = 'Alquiler'
    """
    df = pd.read_sql(sql, conexion)
    conexion.close()
    return df

In [10]:
#Propiedades Vendidas
def listar_propiedades_vendidas():
    conexion = crear_conexion()
    sql = """
    SELECT p.*, t.Nombre_Tipo, e.Nombre_Estado, o.Nombre_Operatoria_Comercial 
    FROM Propiedad p
    JOIN Tipo t ON p.Id_Tipo = t.Id_Tipo
    JOIN Estado e ON p.Id_Estado = e.Id_Estado
    JOIN OperatoriaComercial o ON p.Id_Operatoria_Comercial = o.Id_Operatoria_Comercial
    WHERE e.Nombre_Estado = 'Vendida'
    """
    df = pd.read_sql(sql, conexion)
    conexion.close()
    return df

In [11]:
#Propiedades alquiladas
def listar_propiedades_alquiladas():
    conexion = crear_conexion()
    sql = """
    SELECT p.*, t.Nombre_Tipo, e.Nombre_Estado, o.Nombre_Operatoria_Comercial 
    FROM Propiedad p
    JOIN Tipo t ON p.Id_Tipo = t.Id_Tipo
    JOIN Estado e ON p.Id_Estado = e.Id_Estado
    JOIN OperatoriaComercial o ON p.Id_Operatoria_Comercial = o.Id_Operatoria_Comercial
    WHERE e.Nombre_Estado = 'Alquilada'
    """
    df = pd.read_sql(sql, conexion)
    conexion.close()
    return df

**Uso de las Funciones**

In [12]:
# Listar todas las propiedades
propiedades_totales = listar_propiedades_totales()
print("Listado de propiedades totales:")
print(propiedades_totales)

Listado de propiedades totales:
   Id_Propiedad  Id_Tipo  Id_Estado  Id_Operatoria_Comercial  Id_Propietario  \
0             1        1          1                        1               1   
1             2        1          1                        2               2   
2             3        2          2                        3               3   
3             4        2          1                        1               4   
4             5        3          3                        3               5   
5             6        1          1                        1               1   
6             8        2          1                        1               6   

                     Nombre        Direccion   Contacto   Nombre_Tipo  \
0           Casa a Estrenar     Alberdi 3364  485377855          Casa   
1             Casa Amoblada    Necochea 2325  492178641          Casa   
2  Departamento 2 Ambientes   Rivadavia 4350  478339652  Departamento   
3              Monoambiente   Calch

C:\Users\gorix\AppData\Local\Temp\ipykernel_16984\42421165.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


In [13]:
# Listar propiedades disponibles para la venta
propiedades_venta = listar_propiedades_disponibles_venta()
print("Listado de propiedades disponibles para la venta:")
print(propiedades_venta)

Listado de propiedades disponibles para la venta:
   Id_Propiedad  Id_Tipo  Id_Estado  Id_Operatoria_Comercial  Id_Propietario  \
0             1        1          1                        1               1   
1             4        2          1                        1               4   
2             6        1          1                        1               1   
3             8        2          1                        1               6   

                Nombre       Direccion   Contacto   Nombre_Tipo Nombre_Estado  \
0      Casa a Estrenar    Alberdi 3364  485377855          Casa    Disponible   
1         Monoambiente  Calchines 1722  432578965  Departamento    Disponible   
2     Casa 3 ambientes    Cordoba 3347    4541797          Casa    Disponible   
3  Departamento Grande    Tucuman 3322  485365219  Departamento    Disponible   

  Nombre_Operatoria_Comercial  
0                       Venta  
1                       Venta  
2                       Venta  
3              

C:\Users\gorix\AppData\Local\Temp\ipykernel_16984\461388660.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


In [14]:
# Listar propiedades disponibles para alquiler
propiedades_alquiler = listar_propiedades_disponibles_alquiler()
print("Listado de propiedades disponibles para alquiler:")
print(propiedades_alquiler)

Listado de propiedades disponibles para alquiler:
   Id_Propiedad  Id_Tipo  Id_Estado  Id_Operatoria_Comercial  Id_Propietario  \
0             2        1          1                        2               2   

          Nombre      Direccion   Contacto Nombre_Tipo Nombre_Estado  \
0  Casa Amoblada  Necochea 2325  492178641        Casa    Disponible   

  Nombre_Operatoria_Comercial  
0                    Alquiler  


C:\Users\gorix\AppData\Local\Temp\ipykernel_16984\2418656000.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


In [15]:
# Listar propiedades vendidas
propiedades_vendidas = listar_propiedades_vendidas()
print("Listado de propiedades vendidas:")
print(propiedades_vendidas)

Listado de propiedades vendidas:
   Id_Propiedad  Id_Tipo  Id_Estado  Id_Operatoria_Comercial  Id_Propietario  \
0             3        2          2                        3               3   

                     Nombre       Direccion   Contacto   Nombre_Tipo  \
0  Departamento 2 Ambientes  Rivadavia 4350  478339652  Departamento   

  Nombre_Estado Nombre_Operatoria_Comercial  
0       Vendida               No Disponible  


C:\Users\gorix\AppData\Local\Temp\ipykernel_16984\1061744667.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


In [15]:
# Listar propiedades alquiladas
propiedades_alquiladas = listar_propiedades_alquiladas()
print("Listado de propiedades alquiladas:")
print(propiedades_alquiladas)

Listado de propiedades alquiladas:
   Id_Propiedad  Id_Tipo  Id_Estado  Id_Operatoria_Comercial  Id_Propietario  \
0             5        3          3                        3               5   

           Nombre        Direccion   Contacto Nombre_Tipo Nombre_Estado  \
0  Julian Alvarez  9 de Julio 3572  445683969   Penthouse     Alquilada   

  Nombre_Operatoria_Comercial  
0               No Disponible  


C:\Users\gorix\AppData\Local\Temp\ipykernel_18380\2591256222.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


In [22]:
if __name__ == "__main__": 
    # Ingresar una nueva propiedad y su propietario
    ingresar_propiedad(
        tipo=2,  # Id_Tipo (1-Casa, 2-Departamento, 3-Penthouse, 4-Country, 5-Local Comercial)
        estado=1,  # Id_Estado (1-Disponible, 2- Vendida, 3- Alquilada)
        operativa=1,  # Id_Operatoria_Comercial (1-Venta, 2- Alquiler, 3- No Disponible)
        nombre_propietario='Tomas Castello',  # Nombre del propietario
        direccion_propietario='Cordoba 3347',  # Dirección del propietario
        contacto_propietario='4541231',  # Contacto del propietario
        nombre_propiedad='Departamento Grande',  # Nombre de la propiedad
        direccion_propiedad='Tucuman 3322',  # Dirección de la propiedad
        contacto_propiedad='485365219'  # Contacto de la propiedad
    )

Propiedad ingresada exitosamente con nuevo propietario.


In [ ]:
#Modificar una propiedad
    
# Cambiar solo el nombre y la dirección
modificar_propiedad(1, nombre='Casa Moderna Actualizada', direccion='Nueva Direccion 456')

In [14]:
#Cambiar el estado de la propiedad
modificar_propiedad(3, id_estado=2)

Propiedad 3 modificada exitosamente.


In [ ]:
#Cambiar varios campos a la vez
modificar_propiedad(1, id_estado=2, nombre='Casa Moderna Vendida', id_propietario=3)

In [17]:
#Cambiar la operatoria comercial
modificar_propiedad(3, id_operatoria_comercial=3)

Propiedad 3 modificada exitosamente.


In [28]:
#Eliminar Propiedad
eliminar_propiedad(7) 

Propiedad 7 eliminada exitosamente.
